In [27]:
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages/')

In [1]:
    from venture.shortcuts import *
    import venture.lite.builtin as bp
    import venture.lite.types as t
    import venture.lite.psp as psp
    import numpy as np
    import scipy.stats as sp
    import matplotlib.pyplot as plt
    from util import *
    from collections import defaultdict

%matplotlib inline

In [2]:
v=make_lite_venture_script_ripl()
v.bind_foreign_sp("compute_distance", bp.deterministic_typed(compute_distance2, 
                  [t.NumberType()]*3,t.NumberType()))
v.bind_foreign_sp("compute_azimuth", bp.deterministic_typed(compute_azimuth2, 
                  [t.NumberType()]*3,t.NumberType()))
v.bind_foreign_sp("invert_dist_azimuth2", bp.deterministic_typed(invert_dist_azimuth, 
                  [t.NumberType(),t.NumberType(),t.NumberType()],t.ArrayType(t.NumberType())))

class BoundedExponOutputPSP(psp.RandomPSP):
  # TODO don't need to be class methods
  def simulateNumeric(self,mu,theta,gamma):
    while True:
        x = mu + sp.expon.rvs(scale=1.0/theta)
        if x < gamma:
            return x
  def logDensityNumeric(self,x,mu,theta,gamma):
    if not (mu <= x <= gamma):
        return float('-inf')
    nontailmass = sp.expon.logcdf(gamma - mu,scale=1.0/theta)
    return sp.expon.logpdf(x - mu,scale=1.0/theta) - nontailmass

  def simulate(self,args): return self.simulateNumeric(*args.operandValues())
  def logDensity(self,x,args): return self.logDensityNumeric(x,*args.operandValues())

  def gradientOfLogDensity(self,x,args):
    mu,theta,gamma = args.operandValues()
    nontailmass = sp.expon.cdf(gamma - mu,scale=1.0/theta)
    gradX = -theta
    gradTheta = 1.0 / nontailmass / theta - (x - mu)
    return (gradX,[gradTheta])

  def description(self,name):
    return "  %s(theta) returns a sample from an exponential distribution with rate (inverse scale) parameter theta." % name


v.bind_foreign_sp("bounded_expon", bp.typed_nr(BoundedExponOutputPSP(), [t.NumberType()]*3,t.NumberType()))

## Initialize Priors

In the next block, I initialize all parameters of a physical world description via the prior distributions given in the problem description file. I set the assumptions in my ripl layer while defining the variables locally, then set all local variables into the namedTuple physicsPriors.

In [3]:

lenst = len(STATIONS)

T = v.assume('T','3600.')
R = v.assume('R','6371.')
lambda_e = v.assume('lambda_e','tag("world_params", "lambda_e", gamma(6.0,(4*3.1415927*6371**2*3600)))')
mu_m = v.assume('mu_m','3.0')
theta_m = v.assume('theta_m','4.0')
gamma_m = v.assume('gamma_m','6.0')
v.assume('mu_d_covar','matrix(list(list(13.43,-2.36,-0.0122),list(-2.36,0.452,0.000112),list(-0.0122,0.000112,0.000125)))')
v.assume('mu_a_covar','matrix(list(list(1.23, -0.227, -0.000175),list(-0.227,0.0461,0.0000245),list(-0.000175,0.0000245,0.000000302)))')

v.assume('mu_d','mem( proc(k) { tag("mu_d", k, multivariate_normal(vector(-10.4,3.26,-0.0499),mu_d_covar)) })')
v.assume('mu_t','mem( proc(k) { 0.0 } )')
v.assume('theta_t','mem( proc(k) { tag("theta_t", k, inv_gamma(120,118)) } )')
v.assume('mu_z','mem( proc(k) { 0.0 } )')
v.assume('theta_z','mem( proc(k) { tag("theta_z", k, inv_gamma(5.2,44)) } )')
v.assume('mu_s','mem( proc(k) { 0.0 } )')
v.assume('theta_s','mem( proc(k) { tag("theta_s", k, inv_gamma(6.7,7.5)) } )')
v.assume('mu_a','mem( proc(k) { tag("mu_a", k, multivariate_normal(vector(-7.3,2.03,-0.00196),mu_a_covar)) })')
v.assume('sigma_a','mem( proc(k) { tag("sigma_a", k, inv_gamma(21.1,12.6)**0.5) } )')
v.assume('lambda_f','mem( proc(k) { tag("lambda_f", k, gamma(2.1,1 / 0.0013)) } )')
v.assume('mu_f','mem( proc(k) { tag("mu_f", k, normal(-0.68,0.68)) } )')
v.assume('theta_f','mem( proc(k) { tag("theta_f", k, inv_gamma(23.5,12.45)) } )')

'<procedure>'

In [4]:
def short_phys_real():
    mu_d0 = [-17.359822061671441, -15.768544463698792, -12.922409957811768, -10.773287520963605, -11.132931849171747, -12.477364811457281, -6.7580892401135788, -9.5294379467128927, -11.193494209883042, -9.2582869645622328]
    mu_d1 = [4.4655214067275839, 4.2880673839135159, 3.4939232311517419, 3.0836102728094517, 3.5644521183108333, 3.7061375881329575, 2.9746939897039697, 3.2122834287316029, 3.1747080104009289, 3.0735163035616315]
    mu_d2 = [-0.042587125584878968, -0.046476669950539611, -0.0386056868920115, -0.033644722353157831, -0.058346700937573881, -0.052050312834667954, -0.074411966166429366, -0.057613152060445913, -0.036071371856942244, -0.055272372080290746]
    mu_t = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    theta_t = [0.97403464235339587, 1.0351212113814889, 0.96418499815554193, 1.0079627453372313, 0.84206132362185848, 1.0135927443271529, 1.0186623321740327, 0.95965107920519055, 0.9790147888991162, 0.97492892649666085]
    mu_z = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    theta_z = [18.669473444191421, 31.067605809220918, 7.4717496833821997, 6.8802122424716021, 7.2735470329833278, 14.657952637765698, 6.3892540662578368, 5.0731725691355347, 26.819630675574974, 6.7386639171709755]
    mu_s = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    theta_s = [2.7873617827043962, 2.6365822782429182, 0.71761518576907046, 0.88204931046153978, 1.9977201221848233, 0.94042935154931961, 2.1283948057257942, 1.4122952773438973, 1.6688399530584015, 0.99403597248760345]
    mu_a0 = [-7.5889174296256243, -8.039232467882707, -8.1774264949089801, -5.7676523935644877, -6.8765551696438649, -7.1582798070100742, -8.7307999941883914, -5.7143807424344377, -6.0321268136397386, -8.0335045730895729]
    mu_a1 = [1.9698881472754843, 2.1142324247529105, 2.1359370120152219, 1.7028236897387603, 1.8554536356816058, 1.9893705784398019, 2.2618410084067802, 1.7102856022941844, 1.9118912333453471, 2.2128902234536993]
    mu_a2 = [-0.0017825614388768366, -0.0022352795847745571, -0.0014403097736774737, -0.0024422897060926564, -0.001305072272560544, -0.0024558440921668362, -0.00050303560445385934, -0.0027011475693344989, -0.0021396966460229079, -0.0022089987983199646]
    sigma_a = [0.93195795493497113, 0.70170568773307385, 0.88187075130738513, 0.77700709763472209, 0.96149840769988715, 0.83952976862321427, 0.93385991567304061, 0.72049474691188131, 0.91022810716046143, 0.74539403565284956]
    lambda_f = [0.005356845052932243, 0.004416252349421397, 0.003128792294084219, 0.0018195013716233743, 0.0012844442232585694, 0.001248242568353651, 0.0016220932294251226, 0.001039395307938798, 0.0017931339240377499, 0.0026671057732874284]
    mu_f = [0.6991998411447334, -1.5743137502848807, 0.11003180311800753, -1.5806763114362214, -0.09219581058098525, -0.854561925713898, -0.5313420684257932, -0.8596269262786942, -0.25743993244414687, -0.9563341240466005]
    theta_f = [0.45822676723376737, 0.54164092015782339, 0.56952819745666372, 0.40785210377568526, 0.70803734788388284, 0.47009239496664218, 0.43887083899955331, 0.86740425997874737, 0.48221028166065077, 0.5687694086602948]
    v.assume('sigma_a','mem( proc(k) { (normal(0,1)) } )')
    for i in range(len(STATIONS)):
        v.infer('(observe(mu_d({0}), array({1},{2},{3})))'.format(i,mu_d0[i],mu_d1[i],mu_d2[i]))
        v.infer('(observe(mu_a({0}), array({1},{2},{3})))'.format(i,mu_a0[i],mu_a1[i],mu_a2[i]))
        v.observe('theta_t({0})'.format(i),'{0}'.format(theta_t[i]))
        v.observe('theta_z({0})'.format(i),'{0}'.format(theta_z[i]))
        v.observe('theta_s({0})'.format(i),'{0}'.format(theta_s[i]))
        v.observe('theta_f({0})'.format(i),'{0}'.format(theta_f[i]))
        v.observe('mu_f({0})'.format(i),'{0}'.format(mu_f[i]))
        v.observe('sigma_a({0})'.format(i),'{0}'.format(sigma_a[i]))
        v.observe('lambda_f({0})'.format(i),'{0}'.format(lambda_f[i]))


#short_phys_real()

##Set Model Dependencies from Observations to Background Parameters

In the next block, I set all model dependencies so that my training data will affect the background physics parameters.

In [5]:
#Number of Events in an Episode
v.assume('num_e','mem( proc(epi) {poisson(lambda_e*4*3.1415927*R**2*T)})')

#
#DO I ENCODE UNIFORM DISTRIBUTIONS OF EVENT TIME AND LOCATION?
v.assume('e_t','mem( proc(epi,i) { uniform_continuous(0,T) } )')
v.assume('e_lon','mem ( proc(epi,i) { uniform_continuous(-180,180) } )')
v.assume('sin_e_lat','mem( proc(epi,i) {uniform_continuous(-1,1)})')
#

#Event Magnitude Distribution
v.assume('e_m','mem( proc(epi,i) { bounded_expon(mu_m,1/theta_m,gamma_m) } )')
#Condition Exponential on bounds, using uniform_continuous
#v.assume('obs_func','proc(i){uniform_continuous(mu_m-e_m(epi,i),gamma_m-e_m(epi,i))}')
#v.observe('obs_func(1)','0')

#Detection Probability
v.assume('det_prob','''mem( proc(epi,i,k) {1./(1 + exp(
                                -1. * (vector_dot(mu_d(k),
                                vector(1,e_m(epi,i),compute_distance(k,e_lon(epi,i),sin_e_lat(epi,i)))))))} )
                                ''')
#Travel Time Function
v.assume('I_t','proc(dist) { -0.023 * pow(dist,2) + 10.7 * dist + 5 } ')


#Detection Time
v.assume('Lambda_t','''( proc(epi,i,k) {
         laplace(e_t(epi,i)+I_t(compute_distance(k,e_lon(epi,i),sin_e_lat(epi,i)))+mu_t(k),
         theta_t(k)) } )''')

#Detection Azimuth
v.assume('Lambda_z','''( proc(epi,i,k) { 
          laplace(compute_azimuth(k,e_lon(epi,i),sin_e_lat(epi,i)) - mu_z(k),theta_z(k)) } ) ''' )
#Maybe goes over 360? Problem?

#Slowness Function
v.assume('I_s','proc(dist) { -0.046 * dist + 10.7 } ')
v.assume('inv_I_s','proc(slow) { (slow - 10.7) - 0.046 }')

#Detection Slowness
v.assume('Lambda_s','''( proc(epi,i,k) { laplace(I_s(
         compute_distance(k,e_lon(epi,i),sin_e_lat(epi,i)))+mu_s(k),theta_s(k)) })''')

#Detection Amplitude
v.assume('log_Lambda_a','''( proc(epi,i,k) {normal(vector_dot(mu_a(k),
                                vector(1,e_m(epi,i),I_t(compute_distance(k,e_lon(epi,i),sin_e_lat(epi,i))))),sigma_a(k))})''')

#False Detection Parameters
v.assume('num_Xi','mem( proc(epi,k) {poisson(lambda_f(k) * T)} )')
v.assume('Xi_t','proc(k) {uniform_continuous(0,T)}')
v.assume('Xi_z','proc(k) {uniform_continuous(0,360)}')
v.assume('Xi_s','proc(k) {uniform_continuous(I_s(180),I_s(0))}')
v.assume('log_Xi_a','proc(k) {student_t(1,mu_f(k),theta_f(k))}')


'<procedure>'

In [6]:
episodes = read_episodes('../short_data/training.data')

In [7]:
def check_weight():
    return
    print v.infer('particle_log_weights')[0]
    if v.infer('particle_log_weights')[0] == float('-inf'):
        raise Exception()

In [8]:
for i in range(len(episodes)):
    print 'OBSERVING EPISODE (%s)' % (i)
    v.observe('num_e(%s)'% i,'%s' % len(episodes[i].events))
    check_weight()
    for event_id,event in enumerate(episodes[i].events):
        v.observe('e_t({0},{1})'.format(i,event_id),'%s' % event.time)
        check_weight()
        v.observe('e_lon({0},{1})'.format(i,event_id),'%s' % event.lon)
        check_weight()
        v.observe('sin_e_lat({0},{1})'.format(i,event_id),'%s' % np.sin(np.radians(event.lat)))
        check_weight()
        v.observe('e_m({0},{1})'.format(i,event_id),'%s' % event.mag)
        check_weight()
    for event_id, event_dets in enumerate(episodes[i].assocs):
        for detection_id in event_dets:
            ev = episodes[i].events[event_id]
            det = episodes[i].detections[detection_id]
            v.observe('Lambda_t(%s, %s, %s)' % (i,event_id, det.stanum), '%s' % det.time)
            check_weight()
            v.observe('Lambda_z(%s, %s, %s)' % (i,event_id, det.stanum),'%s' % det.azimuth)
            check_weight()
            v.observe('Lambda_s(%s, %s, %s)' % (i,event_id, det.stanum),'%s' % det.slowness)
            check_weight()
            v.observe('log_Lambda_a(%s, %s, %s)' % (i,event_id, det.stanum),'%s' % np.log(det.amp))
            check_weight()
    station_false_log_amps = defaultdict(list)
    for false_det_id in range(len(sum(episodes[i].assocs,[])),len(episodes[i].detections)):
        false_det = episodes[i].detections[false_det_id]
        station_false_log_amps[false_det.stanum].append(np.log(det.amp))
    for stanum in range(len(STATIONS)):
        v.bulk_observe('log_Xi_a(%s)' % stanum, station_false_log_amps[stanum])
        v.observe('num_Xi(%s, %s)' % (i,stanum), '%s' % len(station_false_log_amps[stanum]))
        check_weight()
        
#v.infer('bogo_possibilize(default, all,1)')

print 'done'

OBSERVING EPISODE (0)
OBSERVING EPISODE (1)
OBSERVING EPISODE (2)
OBSERVING EPISODE (3)
OBSERVING EPISODE (4)
OBSERVING EPISODE (5)
OBSERVING EPISODE (6)
OBSERVING EPISODE (7)
OBSERVING EPISODE (8)
OBSERVING EPISODE (9)
OBSERVING EPISODE (10)
OBSERVING EPISODE (11)
OBSERVING EPISODE (12)
OBSERVING EPISODE (13)
OBSERVING EPISODE (14)
OBSERVING EPISODE (15)
OBSERVING EPISODE (16)
OBSERVING EPISODE (17)
OBSERVING EPISODE (18)
OBSERVING EPISODE (19)
OBSERVING EPISODE (20)
OBSERVING EPISODE (21)
OBSERVING EPISODE (22)
OBSERVING EPISODE (23)
OBSERVING EPISODE (24)
OBSERVING EPISODE (25)
OBSERVING EPISODE (26)
OBSERVING EPISODE (27)
OBSERVING EPISODE (28)
OBSERVING EPISODE (29)
OBSERVING EPISODE (30)
OBSERVING EPISODE (31)
OBSERVING EPISODE (32)
OBSERVING EPISODE (33)
OBSERVING EPISODE (34)
OBSERVING EPISODE (35)
OBSERVING EPISODE (36)
OBSERVING EPISODE (37)
OBSERVING EPISODE (38)
OBSERVING EPISODE (39)
OBSERVING EPISODE (40)
OBSERVING EPISODE (41)
OBSERVING EPISODE (42)
OBSERVING EPISODE (43

In [14]:
print v.infer('global_likelihood'), v.sample('lambda_e')
v.infer('mh("world_params", all, 100)')
print v.infer('global_likelihood'), v.sample('lambda_e')

[-68519.818718894807] 1.92825409256e-12
[-68520.593947723697] 2.06400755396e-12


In [20]:
print [v.sample('theta_z({})'.format(i)) for i in range(len(STATIONS))]
sys.stdout.flush()
v.infer('mh("theta_z", one, 1000)')
print [v.sample('theta_z({})'.format(i)) for i in range(len(STATIONS))]

[18.93317808118974, 36.24370004430702, 17.820979463172915, 5.896856118153806, 13.364220547327758, 31.77361324083149, 15.309996800408417, 4.703259058785331, 38.244869792952315, 8.384024903368964]
[20.75915865077591, 40.61965237011441, 20.353192930363615, 8.432255121755587, 14.308649977283206, 29.795955508458633, 16.70410083807359, 5.092660292793982, 38.244869792952315, 7.242612849770651]


In [13]:
#print v.infer('global_posterior')
#print v.infer('global_likelihood')
v.infer('default_markov_chain(100)')
print v.infer('particle_log_weights')

[-49834.260976841593]


In [12]:
episodes_list = [3]

In [11]:
"""v.assume('real_flip','mem( proc(epi,i,k) {flip(det_prob(epi,i,k) * num_e(epi)/(num_e(epi)+num_Xi(epi,k)))})')
v.assume('poss_e_t','''mem( proc(epi,i,k) {if (real_flip(epi,i,k)) {
         uniform_continuous(0,T) 
         } else {
         uniform_continuous(-1200,T)}} )''')"""

v.assume('poss_e_t','mem( proc(epi,i,k) {uniform_continuous(0,T)})')
v.assume('real_flip','''mem( proc(epi,i,k) {if (poss_e_t(epi,i,k) < 0) {
            False 
            } else {
            flip(det_prob(epi,i,k) * num_e(epi)/(num_e(epi)+num_Xi(epi,k)))}})''')

v.assume('poss_e_lon','mem( proc(epi,i) {uniform_continuous(0,360)})')
v.assume('poss_sin_e_lat','mem( proc(epi,i) { uniform_continuous(-1,1) })')

v.assume('poss_e_fake_travel_time','mem( proc(epi,i,k) {I_t(compute_distance(k,poss_e_lon(epi,i),poss_sin_e_lat(epi,i)))})')
v.assume('poss_e_m','''mem( proc(epi,i,k) {
         if (real_flip(epi,i,k)) {bounded_expon(mu_m,1/theta_m,gamma_m)} else 
         {student_t(1,(mu_f(k) - lookup(mu_a(k),0) - (lookup(mu_a(k),2)*poss_e_fake_travel_time(epi,i,k))) / lookup(mu_a(k),1),theta_f(k)/lookup(mu_a(k),1))}})''')


'<procedure>'

In [12]:
for epi in episodes_list:
    for det_id in range(len(episodes[epi].detections)):
        det = episodes[epi].detections[det_id]
        mu_a = v.sample('mu_a({0})'.format(det.stanum))
        ML_dist = invert_slowness(det.slowness)
        ML_loc = invert_dist_azimuth2(det.stanum,ML_dist,det.azimuth)
        ML_mag = (np.log(det.amp)-mu_a[0]-mu_a[2]*compute_travel_time(ML_dist))/mu_a[1]
        ML_time = det.time-compute_travel_time(ML_dist)
        print ML_time
        v.observe('poss_e_lon({0},{1})'.format(epi,det_id),'{0}'.format(ML_loc[0]))
        v.observe('poss_sin_e_lat({0},{1})'.format(epi,det_id),'{0}'.format(np.sin(np.radians(ML_loc[1]))))
        v.observe('poss_e_t({0},{1},{2})'.format(epi,det_id,det.stanum),'{0}'.format(ML_time))
        v.observe('poss_e_m({0},{1},{2})'.format(epi,det_id,det.stanum),'{0}'.format(ML_mag))

v.infer('bogo_possibilize(default, all,1)')

2583.36533368
Trying to annotate an exception led to:
Traceback (most recent call last):
  File "/Users/acobb/Desktop/programs/Venturecxx/env.d/lib/python2.7/site-packages/venture/ripl/ripl.py", line 144, in execute_instruction
    annotated = self._annotated_error(e, instruction)
  File "/Users/acobb/Desktop/programs/Venturecxx/env.d/lib/python2.7/site-packages/venture/ripl/ripl.py", line 158, in _annotated_error
    exp, text_index = self.humanReadable(**frame)
  File "/Users/acobb/Desktop/programs/Venturecxx/env.d/lib/python2.7/site-packages/venture/ripl/ripl.py", line 385, in humanReadable
    if hasattr(sys.stdout, "fileno") and os.isatty(sys.stdout.fileno()):
  File "/Users/acobb/Desktop/programs/Venturecxx/env.d/lib/python2.7/site-packages/IPython/kernel/zmq/iostream.py", line 205, in fileno
    raise UnsupportedOperation("IOStream has no fileno.")
UnsupportedOperation: IOStream has no fileno.



VentureException: *** evaluation: Cannot find symbol 'bounded_expon'
{'stack_trace': [{'did': 79, 'index': [], 'exp': [{'type': 'symbol', 'value': 'poss_e_m'}, {'type': 'number', 'value': 3}, {'type': 'number', 'value': 0}, {'type': 'number', 'value': 0}]}, {'did': 74, 'index': [1, 2, 1], 'exp': [{'type': 'symbol', 'value': 'mem'}, [{'type': 'symbol', 'value': 'lambda'}, [{'type': 'symbol', 'value': 'epi'}, {'type': 'symbol', 'value': 'i'}, {'type': 'symbol', 'value': 'k'}], [{'type': 'symbol', 'value': 'if'}, [{'type': 'symbol', 'value': 'real_flip'}, {'type': 'symbol', 'value': 'epi'}, {'type': 'symbol', 'value': 'i'}, {'type': 'symbol', 'value': 'k'}], [{'type': 'symbol', 'value': 'bounded_expon'}, {'type': 'symbol', 'value': 'mu_m'}, [{'type': 'symbol', 'value': 'div'}, {'type': 'number', 'value': 1}, {'type': 'symbol', 'value': 'theta_m'}], {'type': 'symbol', 'value': 'gamma_m'}], [{'type': 'symbol', 'value': 'student_t'}, {'type': 'number', 'value': 1}, [{'type': 'symbol', 'value

In [ ]:
compute_travel_time(180)

In [ ]:
v.infer('bogo_possibilize(default, all,1)')

In [ ]:
v.infer('default_markov_chain(1000)')

In [ ]:
print v.infer('particle_log_weights') 

In [ ]:

for j in episodes_list:
    for i,det in enumerate(episodes[j].detections):
        print v.sample('real_flip({0},{1},{2})'.format(j,i,det.stanum))
    print len(episodes[j].detections)
    print v.sample('num_e({0})'.format(j))
    for i in range(len(STATIONS)):
        print v.sample('num_Xi({0},{1})'.format(j,i))

v.assume('assoc_of_det','''mem(proc(epi,det_i,k) {
    if(real_flip(epi,det_i,k)) {
        uniform_discrete(0,num_e(epi))
    } else {
        -1 - det_i
    }
})''')
            

for i in range(1):
    print 'OBSERVING EPISODE (%s)' % (i)
    for det_id, det in enumerate(episodes[i].detections):
        v.observe('Lambda_t({0}, assoc_of_det({0},{1},{2}), {2})'.format(i, det_id, det.stanum), '%s' % det.time)
        v.observe('Lambda_z({0}, assoc_of_det({0},{1},{2}), {2})'.format(i, det_id, det.stanum),'%s' % det.azimuth)
        v.observe('Lambda_s({0}, assoc_of_det({0},{1},{2}), {2})'.format(i, det_id, det.stanum),'%s' % det.slowness)
        v.observe('log_Lambda_a({0}, assoc_of_det({0},{1},{2}), {2})'.format(i,det_id, det.stanum),'%s' % np.log(det.amp))
    

In [ ]:
#v.assume('clusters', 'CLUSTER(filter(real_flip ...,, psudo_e array(poss_e

pseudo_evs = []
for i in episodes_list:
    for det_id,det in enumerate(episodes[i].detections):
         det_flip = v.sample('real_flip({0},{1},{2})'.format(i,det_id,det.stanum))
         if det_flip:
             pseudo_ev_lon = v.sample('poss_e_lon({0},{1})'.format(i,det_id))
             pseudo_ev_lat = np.degrees(np.arcsin(v.sample('poss_sin_e_lat({0},{1})'.format(i,det_id))))
             pseudo_ev_m = v.sample('poss_e_m({0},{1},{2})'.format(i,det_id,det.stanum))
             pseudo_ev_t = v.sample('poss_e_t({0},{1},{2})'.format(i,det_id,det.stanum))
             pseudo_evs.append([pseudo_ev_lon,pseudo_ev_lat,pseudo_ev_m,pseudo_ev_t])
         
for item in pseudo_evs:
    print item